In [ ]:
import pandas as pd
import psycopg2
import pyodbc
import datetime

In [ ]:
warnings.filterwarnings('ignore')

In [ ]:
#Passwords
ONEVIEW_PASSWORD = ''
EDW_USERNAME = ''
EDW_PASSWORD = ''
AIMS_USERNAME = ''
AIMS_PASSWORD = ''

In [ ]:
#OneView connection
conn = psycopg2.connect(
    host='oneview-prd-content-aurora-cluster.cluster-cxgp9osuwqi3.us-east-1.rds.amazonaws.com',
    database='oneview_content',
    user='oneviewadmin',
    password= ONEVIEW_PASSWORD)

In [ ]:
#EDW connection
w = "DSN=prddw; UID={}; PWD={}".format(EDW_USERNAME, EDW_PASSWORD)
AMAEDW = pyodbc.connect(w)

In [ ]:
#Define universe
ov_me_sql = '''
    SELECT medical_education_number FROM ONEVIEW.physician
    '''
ov_me = pd.read_sql_query(ov_me_sql, conn)

In [ ]:
#get party ids
ME_QUERY = \
    """
    SELECT
    P.PARTY_ID,
    P.KEY_VAL AS ME
    FROM
    AMAEDW.PARTY_KEY P
    WHERE
    P.KEY_TYPE_ID = 18
    AND
    P.ACTIVE_IND = 'Y'
    """
party_ids = pd.read_sql(con=AMAEDW, sql=ME_QUERY)

In [ ]:
#get abms
BOARD_QUERY = \
    f"""
    SELECT
    PARTY_ID_FROM,
    PARTY_ID_TO,
    BRD_ID,
    CERTIF_TYPE_CD,
    CERTIF_ID,
    REC_TYPE,
    ISS_DT,
    EXP_DT,
    REVERIFICATION_DT,
    OCCUR_TYPE_CD,
    OCCUR_TYPE_DESC,
    MOC_MET_RQT,
    MOC_PATHWAY_ID,
    ABMS_ID,
    CERT_ID
    FROM
    AMAEDW.BRD_CERTIF
    WHERE
    THRU_DT is null
    """
abms = pd.read_sql(con=AMAEDW, sql=BOARD_QUERY)

In [ ]:
#get license
LIC_QUERY = \
    f"""
    SELECT
    PARTY_ID,
    STATE_ID,
    STS_TYPE_ID,
    LIC_TYPE_ID,
    ISS_DT,
    EXP_DT,
    RNW_DT,
    DEGREE_CD,
    LIC_NBR,
    DELETE_IND
    FROM
    AMAEDW.LIC
    WHERE
    THRU_DT is null
    AND
    DELETE_IND = 'N'
    """
license = pd.read_sql(con=AMAEDW, sql=LIC_QUERY)

In [ ]:
#get dea
DEA_QUERY = \
    f"""
    SELECT
    PARTY_ID,
    DEA_ACTIVITY_CD,
    DEA_SUB_ACTIVITY_CODE,
    SCHED_CD,
    EXP_DT,
    DEA_STATUS_ID,
    DEA_NBR
    FROM
    AMAEDW.DEA_REG
    WHERE
    THRU_DT is null
    AND
    DELETE_IND = 'N'
    """
dea = pd.read_sql(con=AMAEDW, sql=DEA_QUERY)

In [ ]:
#get npi
NPI_QUERY = \
    f"""
    SELECT
    PARTY_ID,
    NPI_CD,
    ENUMERATION_DT,
    DEACTIVATE_DT,
    REACTIVATE_DT
    FROM
    AMAEDW.NPI_REG
    WHERE
    THRU_DT is null
    """
npi = pd.read_sql(con=AMAEDW, sql=NPI_QUERY)

In [ ]:
#merge with pandas
IDS = pd.merge(party_ids, ov_me, left_on='ME', right_on='medical_education_number')
ALL_ABMS = pd.merge(IDS, abms, left_on='PARTY_ID', right_on='PARTY_ID_FROM').drop_duplicates('CERTIF_ID')
ALL_LICENSE = pd.merge(IDS, license, on='PARTY_ID').drop_duplicates()
ALL_LICENSE = ALL_LICENSE.drop_duplicates(['LIC_NBR','DEGREE_CD','STATE_ID','PARTY_ID'])
ALL_DEA = pd.merge(IDS, dea, on='PARTY_ID').drop_duplicates()
ALL_NPI = pd.merge(IDS, npi, on='PARTY_ID').drop_duplicates()